<h1>Importing The Packages</h1>

In [1]:
import numpy as np
#import jax.numpy as npx
#import jax.random as randomx
import pandas as pd
# from dask import dataframe
import os
import time
# import seaborn as sns
# import matplotlib.pyplot as plt
# from matplotlib import rcParams
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import r2_score
import tensorflow as tf
import tensorflow.keras as keras
# from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.cluster import DBSCAN,KMeans
import boruta

<h1>Get The Data</h1>

<h4>I'm treating each investment ID as an investment on a specific company. Thus, Analyse each time series alone.<br/>
Loading the data takes around 5 mins</h4>

In [2]:
t0=time.time()

# dtypes = {
#     'row_id': 'str',
#     'time_id': 'uint16',
#     'investment_id': 'uint32',
#     'target': 'float32',
# }

# for i in range(300):
#     dtypes[f'f_{i}'] = 'float32'

train_df = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')#,usecols=list(dtypes.keys()),dtype=dtypes)

# sup_train_df=pd.read_csv('../input/ubiquant-market-prediction/supplemental_train.csv',usecols=list(dtypes.keys()),dtype=dtypes)

# data_df=pd.concat((train_df,sup_train_df))

# data_df=data_df[data_df['time_id']>600]

print('finished reading the data')

print(f'It took {time.time()-t0} seconds')

finished reading the data
It took 40.050556898117065 seconds


In [3]:
# clustering = KMeans(n_clusters=10,random_state=1,verbose=2)#(eps=5, min_samples=1000,n_jobs=-1)
# clustering.fit(train_df.drop(columns=['row_id','investment_id','time_id']))

In [4]:
# np.unique(clustering.labels_)

In [5]:
# clustering.cluster_centers_

In [6]:
MAX_TIME=train_df['time_id'].max()
del train_df

In [7]:
# n_bins=12

# if n_bins>0:
#     cuts,bins=pd.qcut(train_df['target'],n_bins,labels=[i for i in range(n_bins)],retbins=True)
#     cat_target_column='cat target'
#     train_df[cat_target_column]=cuts.astype(np.int8)
    
#     category_stats={}
#     for cat_target, group in train_df.groupby(cat_target_column):
#         category_stats[cat_target]={}
#         category_stats[cat_target]['mean']=group["target"].mean()
#         category_stats[cat_target]['median']=group["target"].median()
#         category_stats[cat_target]['std']=group["target"].std()

In [8]:
investment_files=[]
investment_path='../input/ubiquant-parquet/investment_ids'
for path, directory, files in os.walk(investment_path):
    investment_files=files

In [9]:
current_time=time.time()
hrs_to_run=3

<h1>Add Lags to the Data</h1>

In [10]:
train_df_lags=pd.DataFrame()
n_lags=10

calculate_lags=False

if calculate_lags:
    iteration=0
    for file_name in investment_files:
        path=os.path.join(investment_path,file_name)
        df=pd.read_parquet(path)

    #     print(len(df))
        df=df.sort_values(by='time_id')

        time_ids=df['time_id'].unique()
        time_min=np.min(time_ids)
        time_max=np.max(time_ids)

        #fill any missing IDs
        n_missing=0
        for i in range(time_min,time_max+1):
            if i not in time_ids:
                n_missing+=1
                df.loc[-i,['time_id']]=i

    #     if n_missing>0:
    #         print(f'there are {n_missing} missing time IDs between {time_min} and {time_max}')

        df['target']=df['target'].fillna(0)

        for shift in range(1,n_lags+1):
            df[f'lag {shift}']=df['target'].shift(shift).fillna(0)

        df=df.dropna()
    #     print(len(df))
        train_df_lags=pd.concat([train_df_lags,df])

        del df

        iteration+=1

        if iteration / 100 == int(iteration // 100):
            print(f'{iteration}th stock was processed')

    train_df_lags.to_parquet('train_df_lags.parquet')

    del train_df_lags

In [11]:
train_df = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')#,usecols=list(dtypes.keys()),dtype=dtypes)
len(train_df)

3141410

<h1>BORUTA</h1>

In [12]:
window=40#240
validation_window=240

scratch=True

#After feature selection i ill train during this time coefficients and then see the performance
#boost on the validation window.
end_time=MAX_TIME-validation_window
start_time=end_time-window

model=RandomForestRegressor(max_depth=100,max_leaf_nodes=12,max_samples=0.7,n_jobs=-1)

features=[f'f_{i}' for i in range(300)]

if (not scratch) and os.path.isfile('../input/ubiquant-feature-selection/boruta_features.csv'):
    # I have to change this filename to the data input location of the new dataset that i will add.
    print('read the present file')
    features_csv=pd.read_csv('../input/ubiquant-feature-selection/boruta_features.csv')
    i=len(features_csv)
else:
    print('generated new DataFrame')
    features_csv=pd.DataFrame(columns=['investment_id']+features)
    i=0

for file in investment_files:
    investment_id=file.split('.')[0]
    
    path=os.path.join(investment_path,file)
    train_df=pd.read_parquet(path)
    
    if int(investment_id) in features_csv['investment_id'].values.astype(int):
        print(investment_id,' skipped')
        continue
    
    #get the Data
#     X_train=train_df[(train_df['time_id']>=start_time) & (train_df['time_id']<=end_time)].fillna(0)#(train_df['time_id']>=start_time) & 
#     X_val=train_df[train_df['time_id']>end_time]
    X_train=train_df
    y_train=X_train['target'].values
#     y_val=X_val['target'].values
#     investment_ids_values_train=X_train['investment_id'].values.reshape(-1,1)
    X_train=X_train[features].values
#     X_val=X_val[features].values
    
#     print('started fitting the model')
    # Train the model
    model.fit(X_train,y_train)
#     print('finished fitting the model')
    
#     print('predict X_val')
#     y_pred=model.predict(X_val)
#     print('finished prediction')
    
#     MSE=mean_squared_error(y_val,y_pred)
    
#     print('MSE: ',MSE)
    # define Boruta feature selection method
    feat_selector = boruta.BorutaPy(model, max_iter=50 ,verbose=0, random_state=1)
    
#     print('started feature selection')
    # find all relevant features
    feat_selector.fit(X_train, y_train)
#     print('finished feature selection')
    
#     X_train=feat_selector.transform(X_train)
    
#     model.fit(X_train,y_train)
    
#     X_val=feat_selector.transform(X_val)
#     print('predict X_val')
#     y_pred=model.predict(X_val)
#     print('finished prediction')
    
#     MSE=mean_squared_error(y_val,y_pred)
    
#     print('after MSE: ',MSE)
    
    # check selected features
#     print(feat_selector.support_)
    
    features_csv.loc[i,['investment_id']]=int(investment_id)
    if True not in feat_selector.support_:
        features_csv.loc[i,features]=[1]*len(feat_selector.support_)
    else:
        features_csv.loc[i,features]=list(map(int,feat_selector.support_))
    
    i+=1

    # check ranking of features
    #print(feat_selector.ranking_)
    
    if (time.time()-current_time) > hrs_to_run*60*60:# after 8 hrs save the progress and leave.
        features_csv.to_csv('boruta_features.csv')
        break
        
    del train_df

features_csv.to_csv('boruta_features.csv')
print('finished')

generated new DataFrame
finished


<h1>RFE</h1>

In [13]:
window=40#240
validation_window=240

#After feature selection i ill train during this time coefficients and then see the performance
#boost on the validation window.
end_time=MAX_TIME-validation_window
start_time=end_time-window

model=RandomForestRegressor(max_depth=100,max_leaf_nodes=12)

features=[f'f_{i}' for i in range(300)]

if os.path.isfile('rfe_features.csv'):
    # I have to change this filename to the data input location of the new dataset that i will add.
    features_csv=pd.read_csv('rfe_features.csv')
else:
    features_csv=pd.DataFrame(columns=['investment_id']+features)

i=0
for file in investment_files:
    
    if (time.time()-current_time)> hrs_to_run*60*60:# after 8 hrs save the progress and leave.
        features_csv.to_csv('rfe_features.csv')
        break
    
    investment_id=file.split('.')[0]
    
    path=os.path.join(investment_path,file)
    train_df=pd.read_parquet(path)
    
    if int(investment_id) in features_csv['investment_id'].values.astype(int):
        print(investment_id,' skipped')
        continue
    
    #get the Data
    X_train=train_df[(train_df['time_id']>=start_time) & (train_df['time_id']<=end_time)].fillna(0)#(train_df['time_id']>=start_time) & 
    X_val=train_df[train_df['time_id']>end_time]
    print(len(X_train))
    
    #X_train=train_df
    y_train=X_train['target'].values
    y_val=X_val['target'].values
    investment_ids_values_train=X_train['investment_id'].values.reshape(-1,1)
    X_train=X_train[features].values
    X_val=X_val[features].values
    
    if len(X_train)==0:
        continue
    print('started fitting the model')
    # Train the model
    model.fit(X_train,y_train)
    print('finished fitting the model')
    
    print('predict X_val')
    y_pred=model.predict(X_val)
    print('finished prediction')
    
    MSE=mean_squared_error(y_val,y_pred)
    
    print('MSE: ',MSE)
    # define Boruta feature selection method
    feat_selector = RFE(model, n_features_to_select=0.2, step=1)
    
    print('started feature selection')
    # find all relevant features
    feat_selector.fit(X_train, y_train)
    print('finished feature selection')
    
    X_train=feat_selector.transform(X_train)
    
    model.fit(X_train,y_train)
    
    X_val=feat_selector.transform(X_val)
    print('predict X_val')
    y_pred=model.predict(X_val)
    print('finished prediction')
    
    MSE=mean_squared_error(y_val,y_pred)
    
    print('after MSE: ',MSE)
    
#     check selected features
    print(feat_selector.support_)
    
    features_csv.loc[i,['investment_id']]=investment_id
    features_csv.loc[i,features]=list(map(int,feat_selector.support_))
    
    i+=1

    # check ranking of features
    #print(feat_selector.ranking_)
    
    if (time.time()-current_time)> 8*60*60:# after 8 hrs save the progress and leave.
        features_csv.to_csv('rfe_features.csv')
        break
        
    del train_df

features_csv.to_csv('rfe_features.csv')
print('finished')

finished


<h1>Auto encoder</h1>

In [14]:
# train_df = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')

# features=train_df.columns[train_df.columns.map(lambda x: 'f' in x)]

# window=120

# X_train=train_df[(train_df['time_id']>MAX_TIME-window) & (train_df['time_id']<=MAX_TIME)]

# del train_df

# dropout_encoder = keras.models.Sequential([
# keras.layers.Flatten(input_shape=[len(features)]),
# keras.layers.Dropout(0.5),
# keras.layers.Dense(100, activation="elu"),
# keras.layers.Dense(5)
# ])
# dropout_decoder = keras.models.Sequential([
# keras.layers.Dense(100, activation="elu", input_shape=[5]),
# keras.layers.Dense(len(features))
# ])
# dropout_ae = keras.models.Sequential([dropout_encoder, dropout_decoder])

In [15]:
# dropout_ae.compile(loss="mae",optimizer=keras.optimizers.Adam())
# dropout_ae.fit(X_train[features].values, X_train[features].values, epochs=10)

In [16]:
# for file_name in investment_files:
#     print(file_name)
#     path=os.path.join(investment_path,file_name)
#     df=pd.read_parquet(path)
    
#     dropout_encoder = keras.models.Sequential([
#     keras.layers.Flatten(input_shape=[len(features)]),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(100, activation="elu"),
#     keras.layers.Dense(300)
#     ])
#     dropout_decoder = keras.models.Sequential([
#     keras.layers.Dense(100, activation="elu", input_shape=[300]),
#     keras.layers.Dense(len(features))
#     ])
#     dropout_ae = keras.models.Sequential([dropout_encoder, dropout_decoder])
    
#     dropout_ae.compile(loss="mae",optimizer=keras.optimizers.Adam())
#     dropout_ae.fit(X_train[features].values, X_train[features].values, epochs=20)
    
#     del df